In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_page_content(url):
    """Verilen URL'den sayfa içeriğini alır."""
    response = requests.get(url)
    return BeautifulSoup(response.content, 'html.parser')

def extract_article_links(soup):
    """Sayfa içeriğinden makale başlıklarını ve bağlantılarını çeker."""
    articles = soup.find_all('h2', class_='entry-title')
    links = []
    titles = []
    
    for article in articles:
        title = article.get_text(strip=True)
        link = article.find_parent('a')['href']
        titles.append(title)
        links.append(link)
        
    return titles, links

def extract_article_content(link):
    """Verilen makale bağlantısından içeriği çeker."""
    article_soup = fetch_page_content(link)
    content_div = article_soup.find('div', class_='entry-content')
    
    if content_div:
        # Paylaşım butonlarını içeren div'i kaldır
        share_buttons = content_div.find('div', class_='share_buttons')
        if share_buttons:
            share_buttons.extract()
        
        # Uyarı metnini içeren div'i kaldır
        warning_block = content_div.find('div', class_='ytd-block')
        if warning_block:
            warning_block.extract()

        # Kalan içeriği al
        return content_div.get_text(strip=True)
    return ""

def scrape_coinkolik(total_pages):
    """Tüm sayfalardan haber başlıklarını ve içeriklerini çeker."""
    all_titles = []
    all_contents = []
    
    for page in range(1, total_pages + 1):
        url = f'https://www.coinkolik.com/kripto-para-haberleri/page/{page}/'
        soup = fetch_page_content(url)
        titles, links = extract_article_links(soup)

        all_titles.extend(titles)

        for link in links:
            content = extract_article_content(link)
            all_contents.append(content)

    return all_titles, all_contents

def save_to_csv(titles, contents, filename):
    """Başlıkları ve içerikleri CSV dosyasına kaydeder."""
    df = pd.DataFrame({
        'Başlık': titles,
        'İçerik': contents
    })
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"{len(titles)} başlık ve içerik başarıyla kaydedildi.")

# Ana işlem
if __name__ == "__main__":
    total_pages = 925
    titles, contents = scrape_coinkolik(total_pages)
    save_to_csv(titles, contents, 'coincolik_scraping.csv')


7395 başlık ve içerik başarıyla kaydedildi.
